In [1]:
import numpy as np
from rtree import index
from sklearn.neighbors import KDTree
from sklearn.preprocessing import normalize
from sklearn.preprocessing import minmax_scale

class Motley():
    def __init__(self, threshold=0.2, alpha=0.1, idx_method="rtree"):
        self.threshold = threshold
        self.alpha = alpha
        self.idx_method = idx_method if idx_method == "kdtree" else "rtree"
    
    # X means data point in the spatial space
    # Z means corresponding attribute representation
    def datafeed(self, X, Z):
        # rownum: number of points, colnum: spatial dimension
        rownum, colnum = X.shape
        
        if(rownum != Z.shape[0]):
            print("Number of input data doesn't match")
            return
        
        self.attributeset = Z
        num_attrs = self.attributeset.shape[1]
        a = self.alpha
        
        # Build up index (kd-tree / R-tree)
        if self.idx_method == "kdtree":
            # Note: kd-tree doesn't support additional object linkage
            self.index = KDTree(X)
        else:
            p = index.Property()
            p.dimension = colnum
            self.index = index.Index(properties=p)
            for idx, row in enumerate(X):
                # (1) Index based on row number
                # (2) Store point for the bounding box
                # (3) Store attribute representation as inner object
                self.index.insert(idx, np.append(row, row), Z[idx])
                
        # Weights used for computing MinDiv
        # Number of weights depends on dimension of attribute space.
        self.weight = np.fromfunction(
            lambda self, x: ((a**(x))*(1-a)/(1-a**num_attrs))
            , (1, num_attrs))
    
    # Query a point and find its diversed neighbors
    # aggress is set for next round's search, if k neighbors are not found
    # max_iter is set to avoid whole-document scanning
    def search(self, qs_space, k=10, aggress=5, approach="greedy", max_iter=5):
        # Initial search: nearest (k * aggress) points
        s_amount = k*aggress
        filtered, num_iter = 0, 0 # Neighbors found / Iteration already run
        
        # Initial result contains zero row, so the nearest neighbor is guaranteed
        # to be in the result set.
        res = np.empty((0, self.attributeset.shape[1]))
        ret = []
        
        if self.idx_method == "kdtree":
            # TODO: should stop if s_amount > size of dataset
            while (len(res) != k) or (num_iter < max_iter):
                # [filtered:] - Exclude those already exaimed
                q_ans = self.index.query(qs_space, k=s_amount, return_distance=False)[filtered:]
                # the query returns a list of indices, get point attributes from self.attrs
                for cand in q_ans:
                    # Add if pass the diversity test
                    if self.diversity_check_greedy(res, self.attributeset[cand]):
                        ret.append(cand)
                        res = np.vstack([res, cand])
                    
                    filtered += 1
                    
                    if len(res) == k:
                        break
                # Start the next round
                if len(res) != k:
                    num_iter += 1
                    s_amount *= aggress
        else:
            while len(res) != k and (num_iter < max_iter):
                print("round %d..." % num_iter)
                print("search size = %d" % s_amount)
                q_ans = self.index.nearest(np.append(qs_space, qs_space), s_amount, objects=True)
                for cand in q_ans:
                    tmp_attr = cand.object
                    if self.diversity_check_greedy(res, tmp_attr):
                        ret.append(cand.id)
                        res = np.vstack([res, tmp_attr])
                    filtered += 1
                    if len(res) == k:
                        break
                        
                if len(res) != k:
                    num_iter += 1
                    s_amount *= aggress
        return ret
    
    def diversity_check_greedy(self, X, q):
        size_data, _ = X.shape

        for i in range(size_data):
            # Sort 1-D difference (In ascending order)
            diff_sorted = np.sort(np.absolute(X[i] - q))
            # Weighting
            divdist_tmp = diff_sorted * self.weight
            # If difference is too small, dispose it
            if divdist_tmp.sum() <= self.threshold:
                return False
        return True

                
                

In [2]:
'''
max_line = 4000
dataset_path = "./../dataset/Forest_Cover/covtype.data"
dataset_w_path = "./../dataset/Forest_Cover/covtype_s.data"
fd = open(dataset_path)
fd_w = open(dataset_w_path, "w")
ct = 0
for line in fd.readlines():
    fd_w.write(line + '\n')
    ct += 1
    if ct >= max_line:
        break
        
fd.close()
fd_w.close()
'''
dataset_path = "./../dataset/Forest_Cover/covtype.data"
r = np.genfromtxt(dataset_path, delimiter=',', dtype=None, names=None)
r.shape

(581012, 55)

In [3]:
data_space = minmax_scale(r[::, (0, 1)], copy=False)
data_attributes = minmax_scale(r[::, (2, 3, 4)], copy=False)

print(data_space.shape)
print(data_attributes.shape)

(581012, 2)
(581012, 3)


C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int32 was converted to float64.
  warnings.warn(msg, DataConversionWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:420: DataConversionWarning: Data with input dtype int32 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


In [4]:
finder = Motley()
finder.datafeed(data_space, data_attributes)

space1_max, space1_min, space2_max, space2_min = data_space[::, 0].max(), data_space[::, 0].min(), data_space[::, 1].max(), data_space[::, 1].min()

rand1 = np.random.uniform(space1_min, space1_max, size=(10, 1))
rand2 = np.random.uniform(space2_min, space2_max, size=(10, 1))

test_input_space = np.hstack([rand1, rand2])

res = []

finder.threshold = 0.05

for i, q in enumerate(test_input_space):
    print("Query no. %d" % (i+1))
    res.append(finder.search(q))

    

Query no. 1
round 0...
search size = 50
round 1...
search size = 250
round 2...
search size = 1250
round 3...
search size = 6250
round 4...
search size = 31250
Query no. 2
round 0...
search size = 50
round 1...
search size = 250
round 2...
search size = 1250
round 3...
search size = 6250
round 4...
search size = 31250
Query no. 3
round 0...
search size = 50
round 1...
search size = 250
round 2...
search size = 1250
round 3...
search size = 6250
round 4...
search size = 31250
Query no. 4
round 0...
search size = 50
round 1...
search size = 250
round 2...
search size = 1250
round 3...
search size = 6250
Query no. 5
round 0...
search size = 50
round 1...
search size = 250
round 2...
search size = 1250
round 3...
search size = 6250
round 4...
search size = 31250
Query no. 6
round 0...
search size = 50
round 1...
search size = 250
round 2...
search size = 1250
round 3...
search size = 6250
Query no. 7
round 0...
search size = 50
round 1...
search size = 250
round 2...
search size = 1250
Que

In [5]:
res

[[251136, 1858, 4559, 246076, 225809, 247214, 279473, 579790, 110606],
 [239279, 239281, 244563, 267792, 259526, 263571, 276645],
 [460095,
  313101,
  407305,
  385480,
  525770,
  447558,
  526721,
  510450,
  552436,
  254503],
 [519081,
  374664,
  339086,
  409662,
  310678,
  229346,
  432456,
  550774,
  334913,
  224569],
 [25999, 319241, 179854, 109389, 448819, 390227, 353833, 203226, 369870],
 [198869,
  242597,
  255700,
  301859,
  550555,
  555207,
  518452,
  437298,
  553451,
  376913],
 [346552,
  353735,
  10557,
  538425,
  351281,
  495802,
  454615,
  359838,
  528076,
  345960],
 [227214, 2651, 222161, 229634, 253784, 255465, 223924, 262116],
 [101509,
  224574,
  156331,
  136193,
  370152,
  404178,
  364644,
  345490,
  458939,
  228855],
 [252118,
  259296,
  132844,
  234377,
  228373,
  226451,
  226660,
  551537,
  315978,
  227899]]

In [7]:
# q_space: query in spatial space, S: returned neighbors
# -space: spatial vector; -attr: attribute vector
from scipy.spatial.distance import cdist, pdist
def evaluation(q_space, S_space, S_attr, metric_s="euclidean", metric_a="cosine"):
    # max_dist
    q_dists = cdist([q], S_space, metric=metric_s)
    avg_query_dist = q_dists.mean()
    max_dist = q_dists.max()
    p_dists = pdist(S_attr, metric=metric_a)
    avg_pairwise_dist = p_dists.mean()
    # Farthest point from query in spatial space / Average distance in spatial space
    # / Average pairwise distance between answers in attribute space
    return max_dist, avg_query_dist, avg_pairwise_dist 

In [10]:
for idx, ans in enumerate(res):
    print(evaluation(test_input_space[idx], data_space[ans], data_attributes[ans]))

(0.83857904430967178, 0.77896951942482262, 0.20255558853420152)
(0.42237510968985326, 0.3276089746780303, 0.12979665021814057)
(0.53914274591865718, 0.51970187161901893, 0.1352733505990826)
(0.18881631495590698, 0.16942592715348861, 0.1381368811209859)
(0.65288686295357334, 0.63767005032083612, 0.16755783066269447)
(0.52226304758730835, 0.50929478442014853, 0.20013262433845821)
(0.73753568476491416, 0.63149652254518496, 0.1209804159022561)
(0.35561843814660282, 0.31763989385951297, 0.16572554029467021)
(0.14697673742588793, 0.135831519691541, 0.14029775070142672)
(0.097504919707735851, 0.032297405504245404, 0.085547496563976866)
